In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_val_predict

In [5]:
import sys
sys.executable

'/Users/jghosh2/opt/anaconda3/envs/py365/bin/python'

In [6]:
#pd.set_option('max_rows',None)

In [7]:
def preprocess_text(message):

    #stopwords
    new_stopwords=['Hi','Hello','Team','Thanks','Hey','regards','please','jira','com','ext','image','Capture']
    stpwrd = nltk.corpus.stopwords.words('english')
    stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]',' ', message.lower())
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    return message


# data import

In [2]:
#read input from historical data into dataframe
df_1 = pd.read_excel('/Users/jghosh2/Documents/my-notebook/L1 triage POC/data/latest.xlsx',sheet_name='data')
df_1['Summary']=df_1['Summary'].astype(str)
df_1['Description']=df_1['Description'].astype(str)
df_1.dropna()
print(df_1.shape)
df_1.head()

(46298, 5)


,Summary,Description,Updated Component,Tier Type 1,Tier Type 2
0,Oracle OTM Access Request,"Sander de Jongh, [Sdejo2@ext.uber.com|mailto:S...",Oracle Transport Management,OTM,OTM
1,PO#467928 line not reflecting in oracle,"Hi Team,\n\n\nWe have received the invoice bel...",Coupa,Coupa,BLANK
2,TM1 Access Request,"Hello Team,_x000D_\n_x000D_\nIâ€™m from the St...",EPM/Hyperion,EPM,TM1 Application Platform
3,Invoice approver issue for EDEN SPRINGS NEDERL...,"Hello Team,\n\n\nFor the attached invoice#66/0...",Oracle Payables,Oracle_ap,BLANK
4,<URGENT> Open PO in Oracle,"Hi Team,\n\n\nPlease open PO 471920 in Oracle ...",Coupa,Coupa,Coupa_po


In [5]:
len(df_1['Updated Component'].value_counts())

31

In [10]:
#read input from historical data into dataframe
#df_2 = pd.read_excel('/Users/jghosh2/Documents/my-notebook/L1 triage POC/data/new.xlsx',sheet_name='data')
#df_2['Summary']=df_2['Summary'].astype(str)
#df_2['Description']=df_2['Description'].astype(str)
#df_2.dropna()
#print(df_2.shape)
#df_2.head()
#df_3=pd.concat(df_1,df_2)

# data preparation

In [12]:
#choose sample data from entire data
data_df = df_1.sample(frac=1, random_state=42)
data_df['combined_text'] = data_df[['Summary','Description']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
data_df['processed_text'] = data_df['combined_text'].map(lambda s:preprocess_text(s)) 
data_df.dropna()
print(data_df.shape)
data_df_component=data_df.drop_duplicates(subset=['processed_text','Updated Component'],keep='first')
data_df_tier_type_1=data_df.drop_duplicates(subset=['processed_text','Tier Type 1'],keep='first')
data_df_tier_type_2=data_df.drop_duplicates(subset=['processed_text','Tier Type 2'],keep='first')
data_df_component = data_df_component.reset_index(drop=True)
data_df_tier_type_1=data_df_tier_type_1.reset_index(drop=True)
data_df_tier_type_2=data_df_tier_type_2.reset_index(drop=True)
print(data_df_component.shape,data_df_tier_type_1.shape,data_df_tier_type_2.shape)

(46298, 7)
(41335, 7) (41487, 7) (41792, 7)


data_df['count']=data_df.groupby('Updated Component')['Updated Component'].transform('count')
data_df_1=data_df[data_df['count']>1000]
data_df_2=data_df[data_df['count']<1000]
data_df_1_undersample=data_df_1.sample(frac=0.7,random_state=1)
data_df_final=pd.concat([data_df_1_undersample,data_df_2])
category_count = data_df_final.groupby(['Updated Component'],sort=False).agg({'Updated Component':'count'})
category_count.rename(columns={'Updated Component':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

In [6]:
category_count = data_df.groupby(['Updated Component'],sort=False).agg({'Updated Component':'count'})
category_count.rename(columns={'Updated Component':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

,count
Updated Component,
Access Requests,6962
Coupa,5711
Oracle Payables,4304
Oracle Administration,4140
B2B,3671
Oracle Transport Management,3247
TrinTech,2949
EPM/Hyperion,2701
Oracle General Ledger,2371


In [7]:
category_count = data_df.groupby(['Tier Type 1'],sort=False).agg({'Tier Type 1':'count'})
category_count.rename(columns={'Tier Type 1':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

,count
Tier Type 1,
Coupa,5392
Coupa_access,4945
B2B,3654
Oracle_ap,3561
Oracle_sysadmin,3552
OTM,3247
TrinTech,2949
Oracle_gl,1909
Oracle_other,1806


In [8]:
category_count = data_df.groupby(['Tier Type 2'],sort=False).agg({'Tier Type 2':'count'})
category_count.rename(columns={'Tier Type 2':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

,count
Tier Type 2,
BLANK,15686
Coupa_access_request,4237
OTM,3188
Coupa_po,2244
B2B_Billing,1921
...,...
Trintech_Trintech_Integration,1
DBA_audit_request,1
Trintech_Convertion_Rates,1


# Training of component

In [13]:
X = data_df_component[['Summary','Description','processed_text']]
y= data_df_component['Updated Component']
# split the cicd data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20,stratify=y, random_state = 42)
print('Training data size: {}'.format(len(X_train)))
print('Test data size: {}'.format(len(X_test)))
model_component= Pipeline([('vect', CountVectorizer(ngram_range=(2,2),max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_component.fit(X_train['processed_text'].values, y_train.values)

Training data size: 33068
Test data size: 8267


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


AttributeError: 'MultinomialNB' object has no attribute 'transform'

In [10]:
# model prediction
result=X_test
y_pred = model_component.predict(X_test['processed_text'].values)

result['original_cat']= y_test.values
result['predicted_cat'] = y_pred

result['prediction_cat_confscore'] = np.round_(np.max(model_component.predict_proba(X_test['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred,y_test),'precision_score':precision_score(y_pred,y_test,average='weighted'),'recall_score':recall_score(y_pred,y_test,average='weighted')
,'f1_score':f1_score(y_pred,y_test,average='weighted')}

result['confusion_matrix'] = str(output)
print(result['confusion_matrix'] [5:6].values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


["{'accuracy': 0.7828716584008709, 'precision_score': 0.802963783118462, 'recall_score': 0.7828716584008709, 'f1_score': 0.7902742702798179}"]


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A va

# trainging of tier type

In [11]:
X = data_df_tier_type_1[['Summary','Description','processed_text']]
y=data_df_tier_type_1['Tier Type 1']
#z= data_df['Tier Type 2']
# split the cicd data into train and test 
X_train_tier, X_test_tier, y_train_tier, y_test_tier = train_test_split(X, y,test_size = .20, random_state = 42)
print('Training data size: {}'.format(len(X_train_tier)))
print('Test data size: {}'.format(len(X_test_tier)))
model_tier_type_1 = Pipeline([('vect', CountVectorizer(ngram_range=(2,2),max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_tier_type_1.fit(X_train_tier['processed_text'].values, y_train_tier.values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Training data size: 33189
Test data size: 8298


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.8,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('mnb',
                 MultinomialNB(alpha=0.05, class_prior=None, fit_prior=False))],
         verbose=False)

In [12]:
# model prediction
result_tier=X_test_tier
y_pred_tier = model_tier_type_1.predict(X_test_tier['processed_text'].values)

result_tier['original_cat']= y_test_tier.values
result_tier['predicted_cat'] = y_pred_tier

result_tier['prediction_cat_confscore'] = np.round_(np.max(model_tier_type_1.predict_proba(X_test_tier['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred_tier,y_test_tier),'precision_score':precision_score(y_pred_tier,y_test_tier,average='weighted'),'recall_score':recall_score(y_pred_tier,y_test_tier,average='weighted')
,'f1_score':f1_score(y_pred_tier,y_test_tier,average='weighted')}

result_tier['confusion_matrix'] = str(output)
print(result_tier['confusion_matrix'] [5:6].values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


["{'accuracy': 0.759701132803085, 'precision_score': 0.7909843773210143, 'recall_score': 0.759701132803085, 'f1_score': 0.7710273280782162}"]


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A va

# Training of tier type 2

In [13]:
X = data_df_tier_type_2[['Summary','Description','processed_text']]
z=data_df_tier_type_2['Tier Type 2']
#z= data_df['Tier Type 2']
# split the cicd data into train and test 
X_train_tier, X_test_tier, z_train_tier, z_test_tier = train_test_split(X, z,test_size = .20, random_state = 42)
print('Training data size: {}'.format(len(X_train_tier)))
print('Test data size: {}'.format(len(X_test_tier)))
model_tier_type_2 = Pipeline([('vect', CountVectorizer(max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_tier_type_2.fit(X_train_tier['processed_text'].values, z_train_tier.values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Training data size: 33433
Test data size: 8359


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.8,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('mnb',
                 MultinomialNB(alpha=0.05, class_prior=None, fit_prior=False))],
         verbose=False)

In [14]:
# model prediction
result_tier=X_test_tier
z_pred_tier = model_tier_type_2.predict(X_test_tier['processed_text'].values)

result_tier['original_cat']= z_test_tier.values
result_tier['predicted_cat'] = z_pred_tier

result_tier['prediction_cat_confscore'] = np.round_(np.max(model_tier_type_2.predict_proba(X_test_tier['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(z_pred_tier,z_test_tier),'precision_score':precision_score(z_pred_tier,z_test_tier,average='weighted'),'recall_score':recall_score(z_pred_tier,z_test_tier,average='weighted')
,'f1_score':f1_score(z_pred_tier,z_test_tier,average='weighted')}

result_tier['confusion_matrix'] = str(output)
print(result_tier['confusion_matrix'] [5:6].values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


["{'accuracy': 0.5980380428280895, 'precision_score': 0.6951301410606741, 'recall_score': 0.5980380428280895, 'f1_score': 0.6325326855486416}"]


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A va

# conf score range

In [ ]:
def confscorerange(x):
    #print(type(x))
    if x>0.90:
        y='90-100'
    elif 0.8<x<0.9:
        y='80-90'
    elif 0.7<x<0.8:
        y='70-80'
    elif 0.6<x<0.7:
        y='60-70'
    elif 0.5<x<0.6:
        y='50-60'
    elif 0.4<x<0.5:
        y='40-50'
    else:
        y='0-40'
    return y
result['range'] = result['prediction_cat_confscore'].map(lambda s:confscorerange(s)) 
(result['range'].value_counts()/8007)*100
result['range'].describe()
classification_report = metrics.classification_report(y_test, y_pred, output_dict=True)
display(pd.DataFrame(classification_report).transpose())

# misclassification

In [ ]:
# check the misclassifications
misclassifications= result.loc[result['original_cat']!=result['predicted_cat']]
misclassifications['predicted_cat'].value_counts()

# model saving and data preparation

In [15]:
import pickle
import joblib
import datetime
# save the model to disk
filename_primary= 'finalized_model_component.sav'
pickle.dump(model_component, open(filename_primary, 'wb'))
filename_secondary= 'finalized_model_tier_type_1.sav'
pickle.dump(model_tier_type_1, open(filename_secondary, 'wb'))
filename_tartiary= 'finalized_model_tier_type_2.sav'
pickle.dump(model_tier_type_2, open(filename_tartiary, 'wb'))

In [16]:
train_size = 0.8
train_end = int(len(data_df)*train_size)
df_train = data_df[:train_end]
df_test = data_df[train_end:]
df_train = df_train[['Summary','Description','Updated Component','Tier Type 1','Tier Type 2']]
df_test = df_test[['Summary','Description','Updated Component','Tier Type 1','Tier Type 2']]
#train_size_cicd=0.02
#train_end_cicd = int(len(data_cicd_final)*train_size_cicd)
#df2_train = data_cicd_final[:train_end_cicd]
#df2_test = data_cicd_final[train_end_cicd:]
#df2_train = df2_train[['Item','Description','establishment_type','target']]
#df2_test = df2_test[['Item','Description','establishment_type','target']]
X_train_save = df_train
X_test_save = df_test
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('df_traintestdata.csv')

# hyperparameter tuning for model optimization

In [18]:
import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'mnb__alpha': [0.01,0.05,0.1,0.2,0.3,0.4,0.5],
}
clf = GridSearchCV(model_component, grid_params,n_jobs=-1,verbose=1)
clf.fit(X_train['processed_text'].values, y_train.values)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   33.4s finished
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Best Score:  0.7821928554187787
Best Params:  {'mnb__alpha': 0.05}


In [ ]:
import joblib
x=joblib.load('finalized_model.sav')

In [ ]:
#read input from historical data into dataframe
df_1= pd.read_csv('/Users/jghosh2/Documents/my-notebook/L1 triage POC/data/L1Triaging_TRack (1).csv', encoding='utf8',engine='python')
#choose sample data from entire data
df_1 = df_1.sample(frac=1, random_state=42)
#fill blanks with ''
df_1 = df_1.fillna('')
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df_1['combined_text'] = df_1[['L1T_TicketSummary']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df_1['processed_text'] = df_1['combined_text'].map(lambda s:preprocess_text(s)) 
print(df_1.shape)
df_1 = df_1.reset_index(drop=True)
y_pred = x.predict(df_1['processed_text'].values)
df_1['prediction_cat'] = y_pred
df_1['prediction_cat_confscore'] =np.round_(np.max(x.predict_proba(df_1['processed_text']), axis=1), decimals=2)
df_1.to_csv('L1Triaging_TicketDetails_after_tagging.csv')